In [1]:
!pip install pysolar
from pysolar.solar import *
from dateutil import tz
import datetime
import pandas as pd
import math

from tqdm import tqdm
tqdm.pandas()

In [2]:
tzone = tz.gettz('Europe/Berlin')

In [3]:
# EXAMPLE:
latitude = 52.55075
longitude = 13.414106

date = datetime.datetime(2018, 1, 1, 2, 0, 0, tzinfo=tzone)
sun_altitude = get_altitude(latitude, longitude, date)
sun_altitude

-53.749102831159014

In [4]:
def get_sun_altitude(year,month,weekday,hour, lat, long):
    # Computes the average sun angle for the for equal weekdays for the respective month
  date_1 = datetime.datetime(year,
                           month,
                           weekday,
                           hour,
                           0, 
                           0, 
                           tzinfo=tzone)
  date_2 = datetime.datetime(year,
                               month,
                               weekday+7,
                               hour,
                               0, 
                               0, 
                               tzinfo=tzone)
  date_3 = datetime.datetime(year,
                               month,
                               weekday+14,
                               hour,
                               0, 
                               0, 
                               tzinfo=tzone)
  date_4 = datetime.datetime(year,
                               month,
                               weekday+21,
                               hour,
                               0, 
                               0, 
                               tzinfo=tzone)
  sun_altitude = get_altitude(lat, long, date_1) + get_altitude(lat, long, date_2) + get_altitude(lat, long, date_3) + get_altitude(lat, long, date_4)
  sun_altitude = sun_altitude/4
  return float(sun_altitude)

In [5]:
# Function TEST: Example
get_sun_altitude(2022,4,1,16,52.509,13.385)

33.868894127442694

In [8]:
# IMPORT

# Negative sample
df_negative_sample = pd.read_csv('../data/processed/negative_sample.csv', index_col=[0])
df_negative_sample

# Positive sample
df_positive_sample = pd.read_csv('../data/processed/positive_sample.csv', index_col=[0])
df_positive_sample

,col_id,segment_id,collision_cnt,objectid,land,district,LOR,street,LOR_ab_2021,year,...,collision,mid_lat,mid_lon,length_m,side_strt,hour_sin,hour_cos,month_sin,month_cos,sun_elevation_angle
0,1,41373.0,1,112705,11,12,12304314.0,Ziekowstraße/Freie Scholle,12500824.0,2018,...,1,13.289303,52.589676,764.663725,0.0,0.136167,-0.990686,5.000000e-01,0.866025,49.945648
1,3,828.0,6,112737,11,7,7020202.0,Volkspark (Rudolf-Wilde-Park),7200308.0,2018,...,1,13.347565,52.482210,141.626937,0.0,-0.997669,-0.068242,5.000000e-01,0.866025,-23.587629
2,4,21397.0,2,112747,11,3,3020209.0,Niederschönhausen,3200206.0,2018,...,1,13.403247,52.583373,96.715676,0.0,-0.816970,-0.576680,5.000000e-01,0.866025,4.163252
3,6,41696.0,1,112792,11,4,4061757.0,Forst Grunewald,4200207.0,2018,...,1,13.248260,52.478908,1965.137281,0.0,-0.997669,-0.068242,5.000000e-01,0.866025,-23.554399
4,7,42255.0,2,112805,11,3,3010104.0,Lietzengraben,3100101.0,2018,...,1,13.427264,52.633692,678.697900,0.0,-0.942261,-0.334880,5.000000e-01,0.866025,-9.590441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52113,38472,21737.0,6,145557,11,1,1044201.0,NaN,1401045.0,2020,...,1,13.378833,52.558438,475.840380,0.0,-0.136167,-0.990686,-2.449294e-16,1.000000,37.531107
52114,38511,2782.0,20,145596,11,1,1044201.0,NaN,1401045.0,2020,...,1,13.382865,52.555996,189.106659,0.0,-0.997669,-0.068242,-2.449294e-16,1.000000,-27.019352
52115,38536,2781.0,21,145621,11,1,1044201.0,NaN,1401045.0,2020,...,1,13.375414,52.556590,225.940036,0.0,0.398401,-0.917211,-2.449294e-16,1.000000,52.571240
52116,38556,19720.0,14,145641,11,3,3061228.0,NaN,3601245.0,2020,...,1,13.432055,52.550425,302.232453,0.0,-0.816970,-0.576680,-2.449294e-16,1.000000,0.730020


In [16]:
# Compute Sun Elevation Angle
# Negative sample
df_negative_sample['sun_elevation_angle'] = df_negative_sample.progress_apply(lambda row:get_sun_altitude(row['year'],row['month'],row['weekday'],row['hour'],row['mid_lat'],row['mid_lon']), axis=1)
df_negative_sample

100%|███████████████████████████████████████████████████████████████████████████| 260702/260702 [1:01:56<00:00, 70.15it/s]


In [ ]:
# Positive sample
df_positive_sample['sun_elevation_angle'] = df_positive_sample.progress_apply(lambda row:get_sun_altitude(row['year'],row['month'],row['weekday'],row['hour'],row['mid_lat'],row['mid_lon']), axis=1)
df_positive_sample

 16%|████████████▍                                                               | 8528/52118 [00:49<04:16, 170.26it/s]

In [18]:
# EXPORT
df_negative_sample.to_csv('../data/processed/negative_sample.csv')
df_positive_sample.to_csv('../data/processed/positive_sample.csv')